In [ ]:
import findspark

findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import *

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
(
spark
    .read
    .csv('title_ratings.tsv', header=True, sep='\t')
    .write
    .format('parquet')
    .mode('overwrite')
    .save('dfratings')
)

In [ ]:
(
spark
    .read
    .csv('title_basics.tsv', header=True, sep='\t')
    .write
    .format('parquet')
    .mode('overwrite')
    .save('dfbasics')
)

In [ ]:
df_basics = spark.read.format('parquet').load('dfbasics')

In [ ]:
df_basics.printSchema()

In [ ]:
df_basics.show(5)

In [ ]:
df_ratings = spark.read.format('parquet').load('dfratings')

In [ ]:
df_ratings.printSchema()

In [ ]:
df_ratings.show(5)

<h3>Respondendo as questões do trabalho prático</h3>

In [ ]:
df_basics.withColumn('startYear', col('startYear').cast('int'))

In [ ]:
df_basics.createOrReplaceTempView('df_basics')

In [ ]:
spark.sql("SELECT DISTINCT titleType FROM df_basics").show()

In [ ]:
spark.sql("SELECT COUNT(DISTINCT primaryTitle) FROM df_basics WHERE startYear = 2015 AND titleType in ('tvMovie', 'movie')").show(5)

In [ ]:
spark.sql('''SELECT genre, COUNT(genre) AS g FROM (SELECT SPLIT(genres,',')[0] AS genre
             FROM   df_basics) group by genre ORDER BY g DESC''').show()

In [ ]:
spark.stop()